In [49]:
import io
import pandas as pd
from pandas import json_normalize

import numpy as np
import json
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

import gspread as gs
from datetime import datetime

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [50]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [51]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery


credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
project_id = 'horego-big-query'

# Make clients.
horego_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import psycopg2.pool
def db_connect_cashbac_nopool(host,database,user,password):
    """
    :return:
    """
    connection = psycopg2.connect(host=host,
                              database=database,
                              user=user,
                              password=password,
                              port=5432)
    return connection

##### UPDATE DATA MALL TO POSTGRESS

In [52]:
%%time
import gspread as gs
# gc = gs.service_account(filename='Horego-GCP-Keys.json')
# sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xj4IXNc0qHtpPhMauK7XiYKxCLnaIqDjxJjV4ttqPkk/edit?usp=sharing')

# worksheet = sheet.worksheet('Type 1 Mall')

# outlet_in_mall_type1 = pd.DataFrame(worksheet.get_all_values())
# outlet_in_mall_type1.columns=outlet_in_mall_type1.iloc[0]
# outlet_in_mall_type1=outlet_in_mall_type1[1:]
# outlet_in_mall_type1=outlet_in_mall_type1[outlet_in_mall_type1.building_name!='']
# outlet_in_mall_type1.columns=['input_date', 'address', 'google_id','place_id', 'building_name']

outlet_in_mall_type1=get_include_restaurant_mall_type1('Horego-GCP-Keys.json')
outlet_in_mall_type1.head()

CPU times: user 193 ms, sys: 55.4 ms, total: 249 ms
Wall time: 3.41 s


,input_date,address,google_id,place_id,building_name
1,2023-11-01,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698bda849c4f6d:0xf2c4df30fefb742f,,Aeon Mall JGC
2,2023-11-01,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b508269f60f:0xddb3a598b6a70a06,,Aeon Mall JGC
3,2023-11-01,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b01fe3552b7:0x642084e0b8e54be1,,Aeon Mall JGC
4,2023-11-01,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b69ef50db21:0xcc40fc04d9b5878d,,Aeon Mall JGC
5,2023-11-01,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b01d76e4003:0x9eba3ec8a0b072d,,Aeon Mall JGC


In [56]:
print(outlet_in_mall_type1['input_date'].unique())

print(outlet_in_mall_type1.building_name.nunique())

['2023-11-01']
116


In [57]:
%%time
list_outlet_update=outlet_in_mall_type1[outlet_in_mall_type1['google_id'].isnull()==False]

list_google_id=''
for google_id in outlet_in_mall_type1[outlet_in_mall_type1.google_id!='']['google_id']:
    list_google_id=list_google_id+"'"+str(google_id)+"',"
list_google_id=list_google_id[:-1]

list_place_id=''
for place_id in outlet_in_mall_type1[outlet_in_mall_type1.place_id!='']['place_id']:
    list_place_id=list_place_id+"'"+str(place_id)+"',"
list_place_id=list_place_id[:-1]

#######################################################################################

query_string=f""" 

SELECT table1.* 
FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results_raw` table1
where true
and
( google_id in ({list_google_id}) or place_id in ({list_google_id}))
and not exists
(
  select 1 
  from `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results` table2
  where table1.google_id=table2.google_id
)

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))
    data=data.drop_duplicates()
    print(len(data))

0
CPU times: user 104 ms, sys: 49 ms, total: 153 ms
Wall time: 4.8 s


In [12]:
query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_results` table1
limit 1

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1


In [58]:
my_schema=[]
for column in df_temp.columns :
    if column in ['working_hours','about']:
        my_schema.append(bigquery.SchemaField(column, "JSON"))
    else:
        my_schema.append(bigquery.SchemaField(column, "STRING"))

In [65]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'
# hot_area=get_hot_area_list(cashbac_key)
# data=cleaning_data_merchants(data)
        
data_location=pd.read_csv('daftar_kota_admin_level_kelurahan_with_lat_long.csv',sep=';')
data_location=data_location[(data_location.latitude<=6) &(data_location.latitude>=-11)&(data_location.longitude>=95)&(data_location.longitude<=141)]

data['scrappe_date'] = datetime.now().strftime('%Y-%m-%d')
data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)

# df_temp=pd.read_csv('results/clear_raw_table_filtered.csv',sep=';', nrows=1)
my_schema=[]
save_columns=[]
for column in df_temp.columns :
    if column in data.columns :
        save_columns.append(column)
        
        if column in ['working_hours','about']:
            my_schema.append(bigquery.SchemaField(column, "JSON"))
        else:
            my_schema.append(bigquery.SchemaField(column, "STRING"))
data[save_columns].to_csv('results/temp_upload_raw.csv',sep=';',index=False)

#############################################################################################
#BQ DATALAKE HOREGO
src_path_file='results/temp_upload_raw.csv'
target_path_file=f"scrapping_results/temp_upload_raw.csv"
bucket_name='horego-bq'
upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

table_schema = "horego_datalake_dev"
table_name = "outcrapper_google_results"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/temp_upload_raw.csv"
create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

#############################################################################################
#############################################################################################
#BQ DATALAKE CASHBAC
bucket_name='cashbac_datalake'
src_path_file='results/temp_upload_raw.csv'
target_path_file=f"scrapping_results/temp_upload_raw.csv"
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "outcrapper_google_results"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/temp_upload_raw.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

#############################################################################################


Uploading results/temp_upload_raw.csv
Path in GCS: horego-bq/scrapping_results/temp_upload_raw.csv
table on : horego_datalake_dev.outcrapper_google_results
Uploading results/temp_upload_raw.csv
Path in GCS: cashbac_datalake/scrapping_results/temp_upload_raw.csv
table on : cashbac_datalake_prod.outcrapper_google_results


In [59]:
outlet_in_mall_type1['input_date']=datetime.now().strftime('%Y-%m-%d')
outlet_in_mall_type1.columns=['input_date', 'address', 'google_id','place_id', 'building_name']
outlet_in_mall_type1.head()

,input_date,address,google_id,place_id,building_name
1,2023-11-08,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698bda849c4f6d:0xf2c4df30fefb742f,,Aeon Mall JGC
2,2023-11-08,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b508269f60f:0xddb3a598b6a70a06,,Aeon Mall JGC
3,2023-11-08,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b01fe3552b7:0x642084e0b8e54be1,,Aeon Mall JGC
4,2023-11-08,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b69ef50db21:0xcc40fc04d9b5878d,,Aeon Mall JGC
5,2023-11-08,"Foodcourt, Lantai 3 - Aeon Mall JGC, Jl. Jkt G...",0x2e698b01d76e4003:0x9eba3ec8a0b072d,,Aeon Mall JGC


In [41]:
outlet_in_mall_type1.to_csv('results/outlet_in_mall_type1.csv',sep=';',index=False)

In [60]:
query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.outlet_in_mall_type1` table1
limit 1

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1


In [43]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

df_temp=pd.read_csv('results/outlet_in_mall_type1.csv',sep=';', nrows=1)
my_schema=[]
for column in df_temp.columns :
    my_schema.append(bigquery.SchemaField(column, "STRING"))

# my_schema=[]
# save_columns=[]
# for column in df_temp.columns :
#     if column in data.columns :
#         save_columns.append(column)
        
#         if column in ['working_hours','about']:
#             my_schema.append(bigquery.SchemaField(column, "JSON"))
#         else:
#             my_schema.append(bigquery.SchemaField(column, "STRING"))
# outlet_in_mall_type1[save_columns].to_csv('results/temp_upload_raw.csv',sep=';',index=False)

src_path_file='results/outlet_in_mall_type1.csv'
target_path_file='scrapping_results/outlet_in_mall_type1.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "outlet_in_mall_type1"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/outlet_in_mall_type1.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

print('')
#############################################################################################

src_path_file='results/outlet_in_mall_type1.csv'
target_path_file='scrapping_results/outlet_in_mall_type1.csv'
bucket_name='horego-bq'
upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

table_schema = "horego_datalake_dev"
table_name = "outlet_in_mall_type1"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/outlet_in_mall_type1.csv"
create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/outlet_in_mall_type1.csv
Path in GCS: cashbac_datalake/scrapping_results/outlet_in_mall_type1.csv
table on : cashbac_datalake_prod.outlet_in_mall_type1

Uploading results/outlet_in_mall_type1.csv
Path in GCS: horego-bq/scrapping_results/outlet_in_mall_type1.csv
table on : horego_datalake_dev.outlet_in_mall_type1


##### UPDATE UNKNOWN BY URL

In [20]:
%%time
import gspread as gs
gc = gs.service_account(filename='Horego-GCP-Keys.json')
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xj4IXNc0qHtpPhMauK7XiYKxCLnaIqDjxJjV4ttqPkk/edit?usp=sharing')

worksheet = sheet.worksheet('Type 2 Mall')

outlet_in_mall_type2 = pd.DataFrame(worksheet.get_all_values())
outlet_in_mall_type2.columns=outlet_in_mall_type2.iloc[0]
outlet_in_mall_type2=outlet_in_mall_type2[1:]
outlet_in_mall_type2=outlet_in_mall_type2[outlet_in_mall_type2.building_name!='']
outlet_in_mall_type2.head()

CPU times: user 183 ms, sys: 63.3 ms, total: 246 ms
Wall time: 2.51 s


,input date,Link,address,building_name,Place Id
1,2023-10-06,https://www.google.com/maps/place/Doner+Kebab+...,"Foodcourt, Lantai 3 - Aeon Mall JGC",Aeon Mall JGC,#N/A
2,2023-10-06,https://www.google.com/maps/place/FANPEKKA+Aeo...,"Foodcourt, Lantai 3 - Aeon Mall JGC",Aeon Mall JGC,ChIJjTe4i_2KaS4RWuMHlDVVgEE
3,2023-10-06,https://www.google.com/maps/place/Cream+Fictio...,"Foodcourt, Lantai 3 - Aeon Mall JGC",Aeon Mall JGC,#N/A
4,2023-10-06,https://www.google.com/maps/place/Waroeng+Baku...,"Foodcourt, Lantai 3 - Aeon Mall JGC",Aeon Mall JGC,ChIJ3U68dAKLaS4Rw5GM7xZvX7U
5,2023-10-06,https://www.google.com/maps/place/Kluckee+Hain...,"Foodcourt, Lantai 3 - Aeon Mall JGC",Aeon Mall JGC,ChIJl2-ky82LaS4RLxBu7rOYwvI


In [16]:
tags=["Merchant List","mall outlets type2"]
limit=0

queries=[]
for url in outlet_in_mall_type2['Link'] :
    queries.append(url)

print(len(queries))

1094


In [41]:
metadata={
  "service_name": "google_maps_service_v2",
  "queries": queries,
  "enrich": False,
  "settings": {
    "output_columns": [],
    "output_extension": "json"
  },
  "tags": tags,
  "categories": [],
  "locations": [],
  "enrichments": ["domains_service"],
  "language": "en",
  "region": "ID",
  "limit": limit,
  "organizationsPerQueryLimit": 500,
  "exactMatch": False,
  "useZipCodes": False,
  "dropDuplicates": True,
  "dropEmailDuplicates": False,
  "ignoreWithoutEmails": False,
  "UISettings": {
    "isCustomQueries": True,
    "isCustomCategories": False,
    "isCustomLocations": False
  },
  "enrichLocations": True
}

In [42]:
import requests
import json

# headers = {'X-API-KEY': 'Z29vZ2xlLW9hdXRoMnwxMTY5MDkxNzg4MjU5ODY5ODE3MzJ8NGU3NzIwMTNlNQ'}#testing
headers = {'X-API-KEY': 'YXV0aDB8NjRjYjJjYjg3ZDQxNWIzM2I0MmY5NGYzfGM0OWZlZWE3ZjI'} #PROD

result_scrapping_task = requests.post("https://api.app.outscraper.com/tasks", data=json.dumps(metadata), headers=headers)
print(result_scrapping_task)
result_scrapping_task=result_scrapping_task.json()
print(result_scrapping_task)

<Response [200]>
{'id': '202310060859100013'}


In [63]:
%%time
import os
from google.cloud import bigquery

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'
subtype_exclude=get_subtype_exclude(horego_key)

scrappe_date='20230801'
        
data_location=pd.read_csv('daftar_kota_admin_level_kelurahan_with_lat_long.csv',sep=';')
data_location=data_location[(data_location.latitude<=6) &(data_location.latitude>=-11)&(data_location.longitude>=95)&(data_location.longitude<=141)]
# data_location=data_location[data_location.Province.str.contains('Jakarta|Jawa|Banten|Yogyakarta')]

path = 'outcrapper_raw/merchant_list_no_filter/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

list_df=[]
for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.DataFrame.from_dict(data)
        data = data[data.place_id!='__NO_PLACE_FOUND__']
        
        data['scrappe_date'] = datetime.strptime(file[:8], '%Y%m%d').strftime('%Y-%m-%d')
        data=cleaning_data_merchants(data)
        # data=data[~data['subtypes'].str.contains('|'.join(subtype_exclude.KEYWORD))]
        # data=filtering_data_merchants(data,'Cashbac-GCP-Keys.json')

        data['hot_area']=data.apply(lambda x:get_hot_area(x.latitude,x.longitude,hot_area),axis=1)
        data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        list_df.append(data)
        # data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        # data.to_excel(f"results/{file.replace('.json','.xlsx')}",index=False, engine='xlsxwriter')

data=pd.concat(list_df)
# data=data[df_temp.columns]
print('DONE')

2023092103554971f0_mall_outlets.json
20230925075008a219_mall_outlets.json
202310060859100013_merchant_list_mall_outlets_type2.json
DONE
CPU times: user 1.9 s, sys: 179 ms, total: 2.08 s
Wall time: 6.45 s


In [64]:
group_schema={}
for i in data.columns:
    if i not in ['place_id', 'google_id']:
        # column_grouped.append(i)
        group_schema[i]='last'

data=data.groupby(['place_id', 'google_id']).agg(group_schema).reset_index()
# data=data[df_temp.columns]
data.keyword=data.keyword.str.lower()

In [65]:
temp=outlet_in_mall_type2[['Link','building_name','address']]
temp.columns=['keyword','Mall','Floor']
temp.keyword=temp.keyword.str.lower()

data=pd.merge(data,temp
              ,on=['keyword']
              ,how='left')

print(data.info())
del temp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Columns: 101 entries, place_id to Floor
dtypes: float64(10), int64(1), object(90)
memory usage: 122.4+ KB
None


In [50]:
# df_temp=pd.read_csv('results/clear_raw_table_filtered.csv',sep=';', nrows=1)

In [70]:
data=data[data.Mall.isnull()==False]

In [66]:
df_temp=pd.read_csv('results/clear_raw_table_filtered.csv',sep=';', nrows=1)
my_schema=[]
save_columns=[]
for column in df_temp.columns :
    if column in data.columns :
        save_columns.append(column)
        
        if column in ['working_hours','about']:
            my_schema.append(bigquery.SchemaField(column, "JSON"))
        else:
            my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].to_csv('results/mall_outlet_by_urls.csv',sep=';',index=False)

In [67]:
from google.cloud import bigquery

data=pd.read_csv('results/mall_outlet_by_urls.csv',sep=';')

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################
# df_temp=pd.read_csv('results/mall_outlet_by_urls.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     if column in ['working_hours','about']:
#         my_schema.append(bigquery.SchemaField(column, "JSON"))
#     else:
#         my_schema.append(bigquery.SchemaField(column, "STRING"))

src_path_file='results/mall_outlet_by_urls.csv'
target_path_file='scrapping_results/mall_outlet_by_urls.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_dev"
table_name = "outcrapper_google_results_mall"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/mall_outlet_by_urls.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema)

#############################################################################################

bucket_name='horego-bq'
src_path_file='results/mall_outlet_by_urls.csv'
target_path_file='scrapping_results/mall_outlet_by_urls.csv'
upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

table_schema = "horego_datalake_dev"
table_name = "outcrapper_google_results_mall"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/mall_outlet_by_urls.csv"
create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema)

Uploading results/mall_outlet_by_urls.csv
Path in GCS: cashbac_datalake/scrapping_results/mall_outlet_by_urls.csv
table on : cashbac_datalake_dev.outcrapper_google_results_mall
Uploading results/mall_outlet_by_urls.csv
Path in GCS: horego-bq/scrapping_results/mall_outlet_by_urls.csv
table on : horego_datalake_dev.outcrapper_google_results_mall


In [68]:
try:
    cursor.close()
    conn_details.close()
except:
    pass
    
conn_details=db_connect_cashbac_nopool('uat-cashbac-db.ciwxizsaix0m.ap-southeast-1.rds.amazonaws.com','cashbacdb','data_ds','data123')
cursor = conn_details.cursor()

for i in tqdm(range(len(data))):
    google_id=data.iloc[i].google_id
    building_name=str(data.iloc[i]['Mall']).upper()
    address=str(data.iloc[i]['Floor']).upper()

    update_query=f"""
        UPDATE cbmerchant.outlet
        SET 
        building_name = '{building_name}'
        ,address='{address}'
        WHERE google_id = '{google_id}'
    """

    cursor.execute(update_query)
    conn_details.commit()

    # pd.DataFrame(columns=['Business Name','Google ID','Mall','Floor']
    #              ,data=[[business_name,google_id,mall,floor]]).to_csv('results/mall_already_input.csv',sep=';',index=False,mode='a',header=False)

cursor.close()
conn_details.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 838/838 [05:28<00:00,  2.55it/s]


In [86]:
temp=outlet_in_mall_type2[['Link','building_name','address']]
temp.columns=['keyword','Mall','Floor']
temp.keyword=temp.keyword.str.lower()

data_left=pd.merge(temp,data
              ,on=['keyword']
              ,how='left')

data_left=data_left[data_left.name.isnull()==True]
data_left=data_left[['keyword','Mall_x','Floor_x']]
data_left.columns=['Link','Mall','Floor']
data_left.to_csv('results/type2_places_not_found.csv',sep=';',index=False)

In [35]:
query_string=f""" 

SELECT google_id
FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results_mall` table1
where true
"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data_type2_on_bq = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

869


In [37]:
# outlet_in_mall_type2=outlet_in_mall_type2[outlet_in_mall_type2['google id'].isin(data_type2_on_bq.google_id)]

In [38]:
tags=["Merchant List","mall outlets type2"]
limit=0

queries=[]
for url in outlet_in_mall_type2['Link'] :
    queries.append(url)

print(len(queries))

1094


In [20]:
hot_area=get_hot_area_list('Horego-GCP-Keys.json')

In [26]:
# df_temp=pd.read_csv('results/clear_raw_table_filtered.csv',sep=';', nrows=1)

In [28]:
%%time
import os
from google.cloud import bigquery

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

scrappe_date='20230801'
        
data_location=pd.read_csv('daftar_kota_admin_level_kelurahan_with_lat_long.csv',sep=';')
data_location=data_location[(data_location.latitude<=6) &(data_location.latitude>=-11)&(data_location.longitude>=95)&(data_location.longitude<=141)]
# data_location=data_location[data_location.Province.str.contains('Jakarta|Jawa|Banten|Yogyakarta')]

path = 'outcrapper_raw/merchant_list_no_filter/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

list_df=[]
for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.DataFrame.from_dict(data)
        data = data[data.place_id!='__NO_PLACE_FOUND__']
        
        data['scrappe_date'] = datetime.strptime(file[:8], '%Y%m%d').strftime('%Y-%m-%d')
        data=cleaning_data_merchants(data)

        data['hot_area']=data.apply(lambda x:get_hot_area(x.latitude,x.longitude,hot_area),axis=1)
        data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        list_df.append(data)
        # data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
        # data.to_excel(f"results/{file.replace('.json','.xlsx')}",index=False, engine='xlsxwriter')

data=pd.concat(list_df)
data=data[df_temp.columns]
print('DONE')

FileNotFoundError: [Errno 2] No such file or directory: 'outcrapper_raw/merchant_list_no_filter/'

In [ ]:
group_schema={}
for i in data.columns:
    if i not in ['place_id', 'google_id']:
        # column_grouped.append(i)
        group_schema[i]='last'

data=data.groupby(['place_id', 'google_id']).agg(group_schema).reset_index()
data=data[df_temp.columns]
data.keyword=data.keyword.str.lower()

In [ ]:
temp=outlet_in_mall[outlet_in_mall['URL'].isnull()==False][['URL','Mall','Floor']]
temp=temp[temp['URL'].str.contains('http')]
temp.columns=['keyword','Mall','Floor']
temp.keyword=temp.keyword.str.lower()

data=pd.merge(data,temp
              ,on=['keyword']
              ,how='left')

del temp

In [ ]:
data.to_csv('results/mall_outlet_by_urls.csv',sep=';',index=False)

In [ ]:
from google.cloud import bigquery

data=pd.read_csv('results/mall_outlet_by_urls.csv',sep=';')

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################
df_temp=pd.read_csv('results/mall_outlet_by_urls.csv',sep=';', nrows=1)
my_schema=[]
for column in df_temp.columns :
    if column in ['working_hours','about']:
        my_schema.append(bigquery.SchemaField(column, "JSON"))
    else:
        my_schema.append(bigquery.SchemaField(column, "STRING"))


src_path_file='results/mall_outlet_by_urls.csv'
target_path_file='scrapping_results/mall_outlet_by_urls.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_dev"
table_name = "outcrapper_google_results_mall"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/mall_outlet_by_urls.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema)

#############################################################################################

bucket_name='horego-bq'
src_path_file='results/mall_outlet_by_urls.csv'
target_path_file='scrapping_results/mall_outlet_by_urls.csv'
upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

table_schema = "horego_datalake_dev"
table_name = "outcrapper_google_results_mall"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/mall_outlet_by_urls.csv"
create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema)

In [3]:
import pandas as pd

In [9]:
import gspread as gs
horego_key='Horego-GCP-Keys.json'
gc = gs.service_account(filename=horego_key)
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1mADtRV9O1WxT-S0D3fG3cnY7k9XbQqDAEbzb_4SGaw0/edit#gid=314789269')
worksheet = sheet.worksheet('Type 1 Mall')
mall_type1 = pd.DataFrame(worksheet.get_all_values())
mall_type1.columns=mall_type1.iloc[0]
mall_type1=mall_type1[1:]

mall_type1=mall_type1[['google id']]
mall_type1.columns=['google_id']
mall_type1=mall_type1[mall_type1.google_id!='']
mall_type1

,google_id
1,0x2e698bda849c4f6d:0xf2c4df30fefb742f
2,0x2e698b508269f60f:0xddb3a598b6a70a06
3,0x2e698b01fe3552b7:0x642084e0b8e54be1
4,0x2e698b69ef50db21:0xcc40fc04d9b5878d
5,0x2e698b01d76e4003:0x9eba3ec8a0b072d
...,...
1848,0x2e6a1dbe2038ef2b:0xf3918c92506c237c
1849,0x2e6a1dbe27361d1f:0x458602678f9771dd
1850,0x2e6a1d73aecb7525:0x2a3b22e34e866034
1851,0x2e6a1d2a9c8f8b6f:0xf202ee9b68428485
